In [ ]:
import os

import matplotlib.pyplot
import modules.preprocessing as preprocess

from PIL import Image, ImageFilter
from tqdm import tqdm
import json

%matplotlib

In [ ]:
from importlib import reload
reload(preprocess)

In [ ]:
# Path

folderPath = "../data/testPolymer/withScotch"
frameToPlot = 250

folder_rawSnapshots = os.path.join(folderPath, "snapshots")
folder_processed_snapshots = os.path.join(folderPath, "snapshots_preProcessed")

assert os.path.isdir(folderPath)
assert os.path.isdir(folder_rawSnapshots), "Could not find raw snapshots"
assert os.path.isdir(folder_processed_snapshots), "Folder preprocessed should be created"

In [ ]:
# Read all images

frame_bounds = [0, 300]

# Sort names
frame_number = (lambda name: int(name[50:-5]))
rawSnapshots_names = sorted([file for file in os.listdir(folder_rawSnapshots) 
                             if file[-5:]=='.tiff'], key=frame_number)

# Only keep frames in bounds
rawSnapshots_names = [name for name in rawSnapshots_names 
                      if (frame_bounds[0]<=frame_number(name)) and (frame_number(name)<=frame_bounds[1])]
rawSnapshots_path = [os.path.join(folder_rawSnapshots, name) for name in rawSnapshots_names]

rawSnapshots_names[:3]

In [ ]:
# Crop

imageRef = Image.open(rawSnapshots_path[frameToPlot])

crop_bounds = preprocess.cropBox_from_GUI(imageRef)
cropped = preprocess.crop_from_coords(imageRef, **crop_bounds)
cropped.show()

print(crop_bounds)

In [ ]:
# Get offsets

n_std_min, n_std_max = 2.5, 2.5

param_contrast = preprocess.offset_scale_from_std(cropped, n_std_min=n_std_min, n_std_max=n_std_max)
contrasted = preprocess.change_intensity(cropped, *param_contrast)
contrasted.show()

In [ ]:
# Crop and contrast every image

nbFrames = len(rawSnapshots_names)

for i in tqdm(range(nbFrames)):
    image = Image.open(rawSnapshots_path[i])
    cropped = preprocess.crop_from_coords(image, **crop_bounds)
    contrasted = preprocess.change_intensity(cropped, *param_contrast)
    contrasted.save(os.path.join(folder_processed_snapshots, f"processed_{rawSnapshots_names[i]}"))


In [ ]:
# Save data

savepath_dict = os.path.join(folder_processed_snapshots, "params_preprocess.txt")

offset_contrast, scale_contrast = param_contrast
dict_data = {"offset_contrast": offset_contrast, "scale_contrast": scale_contrast, 
             "n_std_min": n_std_min, "n_std_max": n_std_max, **crop_bounds}

with open(savepath_dict, 'w') as f:
    f.write(json.dumps(dict_data))